In [1]:
pip install instaloader browser_cookie3


   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ------------ --------------------------- 30.7/99.9 kB 1.3 MB/s eta 0:00:01
   ------------------------------------ --- 92.2/99.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 99.9/99.9 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   --------- ------------------------------ 0.4/1.8 MB 12.9 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 16.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 14.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\huynh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import instaloader
from datetime import datetime
import browser_cookie3
import csv
import time

# The input CSV should have one Instagram profile URL per row.
input_csv = "google (1).csv"

usernames = []

with open(input_csv, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        url = row[0]
        if url.endswith('/'):
            url = url[:-1]
        username = url.split('/')[-1]
        usernames.append(username)

# If header present, skip. Otherwise, comment out the following line.
usernames = usernames[1:]

print(usernames)

# Choose your browser: chrome, firefox, edge, brave, opera, etc.
cookies = browser_cookie3.chrome()  # Change to .firefox(), .edge(), etc. if you use a different browser

# Set up Instaloader and load cookies
L = instaloader.Instaloader()
L.context.session.cookies.update(cookies)

cutoff_date = datetime(2021, 1, 1)

for username in usernames:
    print(f"Downloading posts for: {username}")
    try:
        profile = instaloader.Profile.from_username(L.context, username)
        for post in profile.get_posts():
            if post.date >= cutoff_date:
                L.download_post(post, target=username)
    except Exception as e:
        print(f"Error with {username}: {str(e)}")

['dgphotoholic', 'streetphotographersfdn', 'southstreetsam', 'watchingnewyork', 'asteryx', 'alan_schaller', 'mattu1', 'harebear', 'eortizfoto', 'thesartorialist', 'dino.serrao', 'streetphotographyguy', 'pure.street.photography', 'takeashot_', 'urbanstreetphotography24', 'tavepong_street', 'styleclicker', 'womenstreetphotographers', 'streetwithmatt', 'theguywiththeprinter_', '', 'vineet_vohra', 'nycspc', 'streetphotomag', 'midlifeinparis', 'themonoproject', 'mikeypstreetphotography', 'streetsnappers', 'gokhanarer', 'callmeshlomi', 'streetphotographyindia', 'street.classics', 'pdx.is.cinematic', 'willshooks7', 'streetphotographyinternational', 'dostreetphotography', 'streetphotographybnw', 'karyastreetstyle', 'the-street-photographer', 'streetvisionla', 'boogiephotographer', 'ynotverse', 'photosbyflorentino', 'maciejdakowicz', 'streetphotographerscommunity_', 'monaris_', 'the.street.photography.hub', 'arnold_daniel', 'thewalkingphotographer', 'streetphotographytravlr', 'streetphotography

RequiresAdminError: This operation requires admin. Please run as admin.

In [31]:
import instaloader
import csv
from tqdm import tqdm
import re
import os

def extract_hashtags(caption):
    if not caption:
        return []
    return re.findall(r"#(\w+)", caption)

def remove_hashtags_from_caption(caption):
    # Remove hashtags (words starting with '#') from caption text
    return re.sub(r"#\w+", "", caption).strip()

def download_instagram_posts_to_csv(usernames, max_posts=10, csv_filename="instagram_posts.csv"):
    L = instaloader.Instaloader(download_videos=False, download_comments=False, 
                                save_metadata=False, compress_json=False,
                                quiet=True)
    
    L.load_session_from_file("yuntakesphoto","session-dunghuynh2255")
    posts_data = []
    for username in usernames:
        base_dir = os.path.join("photos", username)
        os.makedirs(base_dir, exist_ok=True)
        profile = instaloader.Profile.from_username(L.context, username)
        posts = profile.get_posts()

        total_processed = 0

        for post in tqdm(posts, desc=f"Processing {username}"):
            if total_processed >= max_posts:
                break

            # Folder per post date
            post_date_str = post.date_utc.strftime("%Y-%m-%d")
            post_dir = os.path.join(base_dir, post_date_str)
            os.makedirs(post_dir, exist_ok=True)

            image_labels = []
            image_counter = 1

            if post.typename == 'GraphSidecar':
                for node in post.get_sidecar_nodes():
                    image_path = os.path.join(post_dir, f"img{image_counter}.jpg")
                    L.download_pic(image_path, node.display_url, post.date_utc)
                    image_labels.append(f"img{image_counter}")
                    image_counter += 1
            else:
                image_path = os.path.join(post_dir, f"img{image_counter}.jpg")
                L.download_pic(image_path, post.url, post.date_utc)
                image_labels.append(f"img{image_counter}")

            caption = post.caption if post.caption else ""
            # Remove hashtags from caption
            caption_no_hashtags = remove_hashtags_from_caption(caption)

            hashtags = extract_hashtags(caption)
            post_url = f"https://www.instagram.com/p/{post.shortcode}/"

            posts_data.append({
                "InstagramLink": post_url,
                "AccountUsername": username,                        # New username column
                "Date": post_date_str,                             # New date column
                "ImageLabels": " | ".join(image_labels),
                "Caption": caption_no_hashtags,                    # Caption without hashtags
                "Likes": post.likes,
                "Comments": post.comments,
                "Hashtags": " | ".join(hashtags)
            })

            total_processed += 1

    # Write CSV with new columns
    with open(csv_filename, mode="w", newline="", encoding="utf-8") as csv_file:
        fieldnames = ["InstagramLink", "AccountUsername", "Date", "ImageLabels", "Caption", "Likes", "Comments", "Hashtags"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(posts_data)


In [33]:
import csv
from urllib.parse import urlparse
import pandas as pd
def get_usernames_by_followers(csv_file_path,min_followers=10000):
    # Read CSV file
    df = pd.read_csv(csv_file_path)

    # Convert followersCount to numeric, coercing errors (in case of bad data)
    df['followersCount'] = pd.to_numeric(df['followersCount'], errors='coerce')

    # Drop rows where followersCount is NaN after conversion
    df = df.dropna(subset=['followersCount'])
    # Filter rows where followersCount is at least min_followers
    df_filtered = df[df['followersCount'] >= min_followers]
    # Sort dataframe by followersCount descending
    df_sorted = df_filtered.sort_values(by='followersCount', ascending=False)

    # Return the list of usernames sorted by followersCount
    return df_sorted['username'].tolist()
# Example usage:
csv_file_path = "profiles.csv"
usernames = get_usernames_by_followers(csv_file_path)
print(usernames)
print(len(usernames))
# usernames=usernames[0:5]
download_instagram_posts_to_csv(usernames, max_posts=15, csv_filename="instagram_posts.csv")

['humansofny', 'stevemccurryofficial', 'sonyalpha', 'jordanmatter', 'streetphotographyinternational', 'dgphotoholic', 'justin.veenema', 'watchingnewyork', 'modelstrangers', 'dino.serrao', 'thesartorialist', 'itsbleg', 'alan_schaller', 'theindependentphoto', 'southstreetsam', 'sutejpannu', 'shotsavant', 'styleclicker', 'streetphotographersfdn', 'edwarnyc', '_.kiruba._', 'monaris_', 'mrstreetpeeper', 'arnold_daniel', 'seantuck', 'yyc_street_photography', 'streetphotographyindia', 'bruce_gilden', 'billydeee', 'asteryx', 'street.classics', 'sixstreetunder', 'vineet_vohra', 'philpenman', 'palebluewave', 'joshkjack', 'underground_nyc', 'bali.sunnypeople', 'ashrafularefin', 'mark.fearnley', 'pure.street.photography', 'stevesweatpants', 'souhayl_a', 'ali_syed84', 'makuto_r50', 'the.street.photography.hub', 'tavepong_street', 'womenstreetphotographers', 'peterturnley', 'urbanstreetphotogallery', 'boogiephotographer', 'snapsbyfox', 'fernando_pedro_salgado', 'theluxographer', 'mattu1', 'streetpho

Processing humansofny: 15it [00:30,  2.03s/it]
Processing stevemccurryofficial: 15it [00:40,  2.73s/it]
Processing sonyalpha: 15it [00:40,  2.69s/it]
Processing jordanmatter: 15it [00:42,  2.82s/it]
Processing streetphotographyinternational: 15it [00:42,  2.82s/it]
Processing dgphotoholic: 15it [00:57,  3.81s/it]
Processing justin.veenema: 15it [00:37,  2.52s/it]
Processing watchingnewyork: 15it [01:09,  4.65s/it]
Processing modelstrangers: 15it [00:45,  3.05s/it]
Processing dino.serrao: 15it [00:46,  3.08s/it]
Processing thesartorialist: 15it [01:24,  5.63s/it]
Processing itsbleg: 15it [00:43,  2.90s/it]
Processing alan_schaller: 15it [00:52,  3.49s/it]
Processing theindependentphoto: 15it [00:36,  2.45s/it]
Processing southstreetsam: 15it [01:16,  5.13s/it]
Processing sutejpannu: 15it [00:46,  3.12s/it]
Processing shotsavant: 15it [00:40,  2.73s/it]
Processing styleclicker: 15it [00:46,  3.09s/it]
Processing streetphotographersfdn: 15it [00:42,  2.84s/it]
Processing edwarnyc: 15it [0